In [ ]:
'''
La derniere étape de notre projet est d'utiliser les modèles de machine learning choisi pour essayer de prédire 
les données de 2022 puis de comparer les écarts avec le reel.
'''

In [3]:
# Étape 1 : Preprocessing et feature engineering de la base de donnée 2022 pour la rendre exploitable.

# Import des modules et fichiers.
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

df_2022_full=pd.read_csv('../data/world-hapiness-report-2022.csv',sep=";")

# Suppression de toutes les lignes non relatives à 2022.
df_2022=df_2022_full[(df_2022_full['year']==2022)]

# Ajout des continents.
df_continents=pd.read_csv('../data/pays&continents.csv',sep=';')
df_2022=df_2022.merge(right=df_continents,on='Country name',how='outer')
df_2022.insert(1,'Regional indicator',df_2022.pop('Regional indicator'))

# Suppression des valeurs manquantes.
df_2022=df_2022.dropna()

# Suppression de la colonne des années.
df_2022=df_2022.drop(['year'],axis=1)

# Copie du jeux de données.
df_2022_continents = df_2022.copy()

# Séparation des variables explicatives et de la variable nom de pays de la variable cible.
X_test2022_continents=df_2022_continents.drop(['Life Ladder', 'Country name'],axis=1)
y_test2022=df_2022_continents['Life Ladder']

#On encode les continents de X_2022
X_test2022_continents=pd.get_dummies(X_test2022_continents,columns=['Regional indicator'])

# Normalisation des données.
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity','Perceptions of corruption','Positive affect','Negative affect']
X_test2022_continents[cols]=scaler.fit_transform(X_test2022_continents[cols])

# Suppression de la colonne "Regional indicator_Central and Eastern Europe" pour correspondre à l'encodage des données
# fait par le OneHotEncoding.
X_test2022_continents=X_test2022_continents.drop(['Regional indicator_Central and Eastern Europe'],axis=1)

# X_test2022_continents et y_test2022 sont alors respectivements les jeux d'entraînement et de test.

In [4]:
# Étape 2 : Récupération des jeux d'entraînement et de test générés dans les notebooks précédents (données de 2006 
# à 2021).

# Import des données.
X_train=pd.read_csv('../data/X_train.csv')
X_test=pd.read_csv('../data/X_test.csv')
y_train=pd.read_csv('../data/y_train.csv')
y_test=pd.read_csv('../data/y_test.csv')

# Concaténation.
X_train2021=pd.concat([X_train,X_test],axis=0)
y_train2021=pd.concat([y_train,y_test],axis=0)

# Transofrmation du jeu y en tableau python.
y_train2021=y_train2021['Life Ladder'].values

# Normalisation des données.
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity', 'Perceptions of corruption','Positive affect','Negative affect']
X_train2021[cols]=scaler.fit_transform(X_train2021[cols])

In [5]:
# Étape 3: Faire tourner le modele de Random forest pour la regression et le tester.

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model.fit(X_train2021,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",rf_model.score(X_test2022_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9859152618884515
Score sur jeu de test 2022 : 0.7948578442285915


In [6]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_continents = rf_model.predict(X_test2022_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_continents=np.round(y_predic2022_continents,2)

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_continents['Predictions']=y_predic2022_continents
df_2022_continents=df_2022_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_continents['Ecart en %']=np.round((df_2022_continents['Predictions']-df_2022_continents['Life Ladder'])/df_2022_continents['Life Ladder']*100,1)

In [7]:
df_2022_continents.head(10)

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.96,-4.8
2,Argentina,Latin America and Caribbean,6.26,6.09,-2.7
3,Armenia,Commonwealth of Independent States,5.38,4.97,-7.6
4,Australia,North America and ANZ,7.04,7.01,-0.4
5,Austria,Western Europe,7.00,6.73,-3.9
6,Bangladesh,South Asia,3.41,4.12,20.8
7,Belgium,Western Europe,6.86,7.01,2.2
8,Benin,Sub-Saharan Africa,4.22,3.92,-7.1
9,Bolivia,Latin America and Caribbean,5.93,5.34,-9.9
10,Botswana,Sub-Saharan Africa,3.44,4.58,33.1


In [8]:
df_2022_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

# On constate que le modèle prédit plutot bien avec toutefois des écarts plus importants pour les valeurs extremes.

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Southeast Asia,5.530000,5.072500,-7.500000
Commonwealth of Independent States,5.592500,5.185000,-7.000000
East Asia,5.973333,5.576667,-6.766667
Latin America and Caribbean,6.211111,5.797222,-6.500000
Central and Eastern Europe,6.044545,5.762727,-4.327273
North America and ANZ,6.907500,6.805000,-1.550000
Western Europe,6.893333,6.805556,-1.316667
South Asia,4.270000,4.163333,1.066667
Sub-Saharan Africa,4.289643,4.266429,2.021429


In [9]:
display(df_2022_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.99,69.8
93,Sierra Leone,Sub-Saharan Africa,2.56,3.54,38.3
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.14,29.0
113,Zimbabwe,Sub-Saharan Africa,3.30,4.14,25.5
66,Malawi,Sub-Saharan Africa,3.36,3.74,11.3
6,Bangladesh,South Asia,3.41,4.12,20.8
10,Botswana,Sub-Saharan Africa,3.44,4.58,33.1
19,Comoros,Sub-Saharan Africa,3.55,3.90,9.9
101,Tanzania,Sub-Saharan Africa,3.62,4.12,13.8
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.92,-15.9
22,Costa Rica,Latin America and Caribbean,7.08,6.67,-5.8
64,Luxembourg,Western Europe,7.23,6.92,-4.3
83,Norway,Western Europe,7.30,7.43,1.8
78,Netherlands,Western Europe,7.39,7.06,-4.5
98,Sweden,Western Europe,7.43,7.49,0.8
45,Iceland,Western Europe,7.45,7.41,-0.5
26,Denmark,Western Europe,7.55,7.42,-1.7
50,Israel,Middle East and North Africa,7.66,6.82,-11.0
33,Finland,Western Europe,7.73,7.51,-2.8


In [10]:
display(df_2022_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.99,69.8
93,Sierra Leone,Sub-Saharan Africa,2.56,3.54,38.3
10,Botswana,Sub-Saharan Africa,3.44,4.58,33.1
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.14,29.0
113,Zimbabwe,Sub-Saharan Africa,3.30,4.14,25.5
6,Bangladesh,South Asia,3.41,4.12,20.8
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4
101,Tanzania,Sub-Saharan Africa,3.62,4.12,13.8
66,Malawi,Sub-Saharan Africa,3.36,3.74,11.3
39,Ghana,Sub-Saharan Africa,4.19,4.64,10.7


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.31,-25.8
42,Guinea,Sub-Saharan Africa,5.32,4.13,-22.4
77,Nepal,South Asia,5.47,4.27,-21.9
80,Nicaragua,Latin America and Caribbean,6.39,5.36,-16.1
30,El Salvador,Latin America and Caribbean,6.49,5.45,-16.0
63,Lithuania,Central and Eastern Europe,7.04,5.92,-15.9
112,Vietnam,Southeast Asia,6.27,5.28,-15.8
71,Mexico,Latin America and Caribbean,7.04,5.95,-15.5
73,Mongolia,East Asia,5.79,4.90,-15.4
92,Senegal,Sub-Saharan Africa,4.91,4.18,-14.9


In [11]:
'''
On pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele 
vers une réponse plus cohérente en fonction du continents. 
On réalise donc un essai sur les mêmes données amputées de la variable continent.
'''

"\nOn pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele \nvers une réponse plus cohérente en fonction du continents. \nOn réalise donc un essai sur les mêmes données amputées de la variable continent.\n"

In [12]:
# Séparation des variables explicatives de la variable cible et suppression des noms de pays.
X_test2022_no_continents=df_2022.drop(['Life Ladder','Country name','Regional indicator'],axis=1)
y_test2022_no_continents=df_2022['Life Ladder']

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_test2022_no_continents[cols]=scaler.fit_transform(X_test2022_no_continents[cols])

#X_test2022_no_continents et y_test2022_no_continents sont alors respectivements nos variables explicatives et notre variable cible de notre jeu de test

In [13]:
# Suppression variables des continents du jeu de données 2021.
X_train2021_no_continents=X_train2021.drop(["Regional indicator_Commonwealth of Independent States"
                              ,"Regional indicator_East Asia"
                              ,"Regional indicator_Latin America and Caribbean"
                              ,"Regional indicator_Middle East and North Africa"
                              ,"Regional indicator_North America and ANZ"
                              ,"Regional indicator_South Asia"
                              ,"Regional indicator_Southeast Asia"
                              ,"Regional indicator_Sub-Saharan Africa"
                              ,"Regional indicator_Western Europe"],axis=1)

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_train2021_no_continents[cols]=scaler.fit_transform(X_train2021_no_continents[cols])

In [14]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model.fit(X_train2021_no_continents,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model.score(X_train2021_no_continents,y_train2021))
print("Score sur jeu de test 2022 :",rf_model.score(X_test2022_no_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9835743248027544
Score sur jeu de test 2022 : 0.7469354141450727


In [15]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_no_continents = rf_model.predict(X_test2022_no_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_no_continents=np.round(y_predic2022_no_continents,2)

# Copie du jeu de donnée 2022 et suppression des continents.
df_2022_no_continents = df_2022.copy()

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_no_continents['Predictions']=y_predic2022_no_continents
df_2022_no_continents=df_2022_no_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_no_continents['Ecart en %']=np.round((df_2022_no_continents['Predictions']-df_2022_no_continents['Life Ladder'])/df_2022_no_continents['Life Ladder']*100,1)

In [16]:
df_2022_no_continents

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6
2,Argentina,Latin America and Caribbean,6.26,6.07,-3.0
3,Armenia,Commonwealth of Independent States,5.38,4.94,-8.2
4,Australia,North America and ANZ,7.04,6.89,-2.1
5,Austria,Western Europe,7.00,6.73,-3.9
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.21,-7.2
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.22,-13.3
112,Vietnam,Southeast Asia,6.27,5.25,-16.3


In [17]:
df_2022_no_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

#Grâce à ce df on s'aperçoit que notre modele prédit plutot bien, il a un peu plus de mal a prédire les valeurs extremes

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Latin America and Caribbean,6.211111,5.546667,-10.594444
Commonwealth of Independent States,5.592500,5.100000,-8.612500
Southeast Asia,5.530000,5.122500,-6.500000
East Asia,5.973333,5.620000,-6.033333
Central and Eastern Europe,6.044545,5.782727,-4.054545
North America and ANZ,6.907500,6.705000,-2.975000
Western Europe,6.893333,6.759444,-1.961111
South Asia,4.270000,4.130000,0.000000
Sub-Saharan Africa,4.289643,4.262143,1.685714


In [18]:
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.92,66.8
93,Sierra Leone,Sub-Saharan Africa,2.56,3.40,32.8
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.06,26.5
113,Zimbabwe,Sub-Saharan Africa,3.30,4.16,26.1
66,Malawi,Sub-Saharan Africa,3.36,3.73,11.0
6,Bangladesh,South Asia,3.41,3.92,15.0
10,Botswana,Sub-Saharan Africa,3.44,4.61,34.0
19,Comoros,Sub-Saharan Africa,3.55,3.84,8.2
101,Tanzania,Sub-Saharan Africa,3.62,4.16,14.9
32,Ethiopia,Sub-Saharan Africa,3.63,4.25,17.1


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.94,-15.6
22,Costa Rica,Latin America and Caribbean,7.08,6.65,-6.1
64,Luxembourg,Western Europe,7.23,6.88,-4.8
83,Norway,Western Europe,7.30,7.41,1.5
78,Netherlands,Western Europe,7.39,7.06,-4.5
98,Sweden,Western Europe,7.43,7.31,-1.6
45,Iceland,Western Europe,7.45,7.39,-0.8
26,Denmark,Western Europe,7.55,7.30,-3.3
50,Israel,Middle East and North Africa,7.66,6.57,-14.2
33,Finland,Western Europe,7.73,7.39,-4.4


In [19]:
display(df_2022_no_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_no_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.92,66.8
10,Botswana,Sub-Saharan Africa,3.44,4.61,34.0
93,Sierra Leone,Sub-Saharan Africa,2.56,3.40,32.8
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.06,26.5
113,Zimbabwe,Sub-Saharan Africa,3.30,4.16,26.1
32,Ethiopia,Sub-Saharan Africa,3.63,4.25,17.1
6,Bangladesh,South Asia,3.41,3.92,15.0
101,Tanzania,Sub-Saharan Africa,3.62,4.16,14.9
39,Ghana,Sub-Saharan Africa,4.19,4.68,11.7
66,Malawi,Sub-Saharan Africa,3.36,3.73,11.0


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.32,-25.6
30,El Salvador,Latin America and Caribbean,6.49,4.97,-23.4
71,Mexico,Latin America and Caribbean,7.04,5.41,-23.2
80,Nicaragua,Latin America and Caribbean,6.39,4.94,-22.7
42,Guinea,Sub-Saharan Africa,5.32,4.13,-22.4
77,Nepal,South Asia,5.47,4.27,-21.9
37,Georgia,Commonwealth of Independent States,5.29,4.26,-19.5
43,Honduras,Latin America and Caribbean,5.93,4.78,-19.4
41,Guatemala,Latin America and Caribbean,6.15,4.99,-18.9
9,Bolivia,Latin America and Caribbean,5.93,4.95,-16.5


In [20]:
# Pour terminer, on aggrège les deux jeux de données.
df_2022 = df_2022_continents
df_2022

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.96,-4.8
2,Argentina,Latin America and Caribbean,6.26,6.09,-2.7
3,Armenia,Commonwealth of Independent States,5.38,4.97,-7.6
4,Australia,North America and ANZ,7.04,7.01,-0.4
5,Austria,Western Europe,7.00,6.73,-3.9
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.17,-7.8
109,Uruguay,Latin America and Caribbean,6.67,6.32,-5.2
110,Uzbekistan,Commonwealth of Independent States,6.02,5.33,-11.5
112,Vietnam,Southeast Asia,6.27,5.28,-15.8


In [21]:
# Changement de nom des colonnes
new_names =  {'Predictions' : 'Prédictions avec continents',
              'Ecart en %'  : 'Écart en % avec continents'}

df_2022 = df_2022.rename(new_names, axis = 1)
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents
1,Albania,Central and Eastern Europe,5.21,4.96,-4.8
2,Argentina,Latin America and Caribbean,6.26,6.09,-2.7
3,Armenia,Commonwealth of Independent States,5.38,4.97,-7.6
4,Australia,North America and ANZ,7.04,7.01,-0.4
5,Austria,Western Europe,7.00,6.73,-3.9
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.17,-7.8
109,Uruguay,Latin America and Caribbean,6.67,6.32,-5.2
110,Uzbekistan,Commonwealth of Independent States,6.02,5.33,-11.5
112,Vietnam,Southeast Asia,6.27,5.28,-15.8


In [22]:
df_2022['Prédictions sans continents'] = df_2022_no_continents['Predictions']
df_2022['Écart en % sans continents'] = df_2022_no_continents['Ecart en %']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents
1,Albania,Central and Eastern Europe,5.21,4.96,-4.8,4.92,-5.6
2,Argentina,Latin America and Caribbean,6.26,6.09,-2.7,6.07,-3.0
3,Armenia,Commonwealth of Independent States,5.38,4.97,-7.6,4.94,-8.2
4,Australia,North America and ANZ,7.04,7.01,-0.4,6.89,-2.1
5,Austria,Western Europe,7.00,6.73,-3.9,6.73,-3.9
...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.17,-7.8,6.21,-7.2
109,Uruguay,Latin America and Caribbean,6.67,6.32,-5.2,6.33,-5.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.33,-11.5,5.22,-13.3
112,Vietnam,Southeast Asia,6.27,5.28,-15.8,5.25,-16.3


In [23]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['Différences valeurs écarts'] = df_2022['Prédictions sans continents']-df_2022['Prédictions avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts
1,Albania,Central and Eastern Europe,5.21,4.96,-4.8,4.92,-5.6,-0.04
2,Argentina,Latin America and Caribbean,6.26,6.09,-2.7,6.07,-3.0,-0.02
3,Armenia,Commonwealth of Independent States,5.38,4.97,-7.6,4.94,-8.2,-0.03
4,Australia,North America and ANZ,7.04,7.01,-0.4,6.89,-2.1,-0.12
5,Austria,Western Europe,7.00,6.73,-3.9,6.73,-3.9,0.00
...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.17,-7.8,6.21,-7.2,0.04
109,Uruguay,Latin America and Caribbean,6.67,6.32,-5.2,6.33,-5.1,0.01
110,Uzbekistan,Commonwealth of Independent States,6.02,5.33,-11.5,5.22,-13.3,-0.11
112,Vietnam,Southeast Asia,6.27,5.28,-15.8,5.25,-16.3,-0.03


In [24]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['% variations écarts'] = df_2022['Écart en % sans continents']-df_2022['Écart en % avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
1,Albania,Central and Eastern Europe,5.21,4.96,-4.8,4.92,-5.6,-0.04,-0.8
2,Argentina,Latin America and Caribbean,6.26,6.09,-2.7,6.07,-3.0,-0.02,-0.3
3,Armenia,Commonwealth of Independent States,5.38,4.97,-7.6,4.94,-8.2,-0.03,-0.6
4,Australia,North America and ANZ,7.04,7.01,-0.4,6.89,-2.1,-0.12,-1.7
5,Austria,Western Europe,7.00,6.73,-3.9,6.73,-3.9,0.00,0.0
...,...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.17,-7.8,6.21,-7.2,0.04,0.6
109,Uruguay,Latin America and Caribbean,6.67,6.32,-5.2,6.33,-5.1,0.01,0.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.33,-11.5,5.22,-13.3,-0.11,-1.8
112,Vietnam,Southeast Asia,6.27,5.28,-15.8,5.25,-16.3,-0.03,-0.5


In [25]:
df_2022.describe()

,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,5.591359,5.406408,-1.766019,5.343883,-2.873786,-0.062524,-1.107767
std,1.229449,1.063460,12.810302,1.058889,12.967833,0.155801,2.731971
min,2.350000,3.540000,-25.800000,3.400000,-25.600000,-0.670000,-12.100000
25%,4.675000,4.400000,-8.450000,4.420000,-9.350000,-0.120000,-2.350000
50%,5.890000,5.360000,-3.900000,5.240000,-4.000000,-0.030000,-0.500000
75%,6.465000,6.145000,1.500000,6.195000,1.250000,0.030000,0.600000
max,7.730000,7.510000,69.800000,7.410000,66.800000,0.500000,8.700000


In [26]:
# On constate que les prédictions sont moins précises si l'on enlève les données relatives aux continents.

In [27]:
# Séparation des écarts en positifs et négatifs
ecarts_positifs = df_2022[df_2022['Écart en % avec continents'] > 0]['Écart en % avec continents']
ecarts_negatifs = df_2022[df_2022['Écart en % avec continents'] < 0]['Écart en % avec continents']

# Calcul de la moyenne et de la médiane pour les écarts positifs
ecart_positif_moyen = ecarts_positifs.mean()
ecart_positif_median = ecarts_positifs.median()

# Calcul de la moyenne et de la médiane pour les écarts négatifs
ecart_negatif_moyen = ecarts_negatifs.mean()
ecart_negatif_median = ecarts_negatifs.median()

# Affichage des résultats
print("Écart négatif moyen:", ecart_negatif_moyen)
print("Écart positif moyen:", ecart_positif_moyen)
print("Écart négatif médian:", ecart_negatif_median)
print("Écart positif médian:", ecart_positif_median)

Écart négatif moyen: -7.725714285714285
Écart positif moyen: 10.875757575757575
Écart négatif médian: -6.300000000000001
Écart positif médian: 5.2


In [32]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calcul de l'erreur quadratique moyenne (MSE) et de la racine carrée de l'erreur quadratique moyenne (RMSE)
# avzec les continents.
mse_continents = mean_squared_error(y_test2022, y_predic2022_continents)
rmse_continents = np.sqrt(mse_continents)

# Affichage des résultats
print(f"Erreur quadratique moyenne avec les continents (MSE) : {mse_continents}")
print(f"Racine carrée de l'erreur quadratique moyenne avec les continents (RMSE) : {rmse_continents}")

# Calcul de l'erreur quadratique moyenne (MSE) et de la racine carrée de l'erreur quadratique moyenne (RMSE) 
# sans les continents.
mse_no_continents = mean_squared_error(y_test2022, y_predic2022_no_continents)
rmse_no_continents = np.sqrt(mse_no_continents)

# Affichage des résultats
print(f"Erreur quadratique moyenne sans les continents (MSE) : {mse_no_continents}")
print(f"Racine carrée de l'erreur quadratique moyenne sans les continents (RMSE) : {rmse_no_continents}")

Erreur quadratique moyenne avec les continents (MSE) : 0.30744951456310676
Racine carrée de l'erreur quadratique moyenne avec les continents (RMSE) : 0.5544813022664576
Erreur quadratique moyenne sans les continents (MSE) : 0.3786708737864078
Racine carrée de l'erreur quadratique moyenne sans les continents (RMSE) : 0.6153623922424962


In [ ]:
# On constate que la RMSE augmente sans les continents. Le modèles est donc bien moins performant.